## Data prep

In [4]:
# Import packages and load the data
import numpy as np
import scipy.special as spc
import numpy.linalg as lin
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib notebook

In [6]:
macro = np.loadtxt(
    'C:/Users/Hrisi/Structural_estimation/StructEst_W18/ProblemSets/PS5/NewMacroSeries.txt', delimiter=",")

In [7]:
c=macro[:,0]
k=macro[:,1]
w=macro[:,2]
rate=macro[:,3]
y=macro[:,4]

In [10]:
b=0.99
k_1=np.mean(k)
print(k_1)

6643985.1383


In [262]:
# simulate 100-dimensional unifirm vectors, 1000 times
T = 100
S = 1000
unif_vals = sts.uniform.rvs(0, 1, size=(T, S))
print(unif_vals.shape)


(100, 1000)


In [335]:
def model_moments (unif_vals, alpha, rho, mu, sigma):
    dim = np.shape(unif_vals)[1]
    beta=0.99
    e = sts.norm.ppf(unif_vals, loc=0, scale=sigma)
    k_1=6643985.1383
    z_0=mu
    
    z=[]
    k=[]
    w=[]
    rate=[]
    c=[]
    y=[]
    
    # initialize all vectors for t=1
    k.append(e[0]*0+k_1)
    z.append(rho*z_0+(1-rho)*mu+e[0])
    w.append((1-alpha) * np.exp(z[0]) * k[0]**alpha)
    rate.append(alpha* np.exp(z[0])* k[0]**(alpha-1))
    y.append(np.exp(z[0])*k[0]**alpha)
    
    
    # calculate all values for t=2:100, for consumption they go t=1:99
    for t in range(1, 100):
        z.append(rho*z[t-1]+(1-rho)*mu+e[t])
        k.append(alpha*beta*np.exp(z[t-1])*k[t-1]**alpha)
        w.append((1-alpha) * np.exp(z[t]) * k[t]**alpha)
        rate.append(alpha* np.exp(z[t])* k[t]**(alpha-1))
        y.append(np.exp(z[t])*k[t]**alpha)
        c.append(rate[t-1]*k[t-1]+w[t-1]-k[t])
    
    #Calculate k_101 and c_100
    k.append(alpha*beta*np.exp(z[t-1])*k[t-1]**alpha)
    c.append(rate[t-1]*k[t-1]+w[t-1]-k[t])
    
    # Calculate moments
    m1 = np.mean(np.array(c))
    m2 = np.mean(np.array(k[0:100]))
    m3 = np.mean(np.array(c)/np.array(y))
    m4 = np.mean(np.std(np.array(y), axis=0)**2)
    m5 = np.trace((np.corrcoef(np.array(c[0:99]).T,np.array(c[1:100]).T))[0:dim,dim:dim*2])/dim
    m6 = np.trace(np.corrcoef(np.array(c[0:100]).T,np.array(k[0:100]).T)[0:dim,dim:dim*2])/dim
    
    m=np.array([m1, m2, m3, m4, m5, m6])
   
    return m

In [336]:
model_moments(unif_vals, 0.45,  0.9 ,  9.2 ,  0.05)

array([  5.37161932e+06,   4.33894123e+06,   5.54512890e-01,
         2.81393470e+12,   9.33988204e-01,   9.18555235e-01])

In [337]:
def data_moments(data):
    c=macro[:,0]
    k=macro[:,1]
    w=macro[:,2]
    rate=macro[:,3]
    y=macro[:,4]
    
    m1 = np.mean(np.array(c))
    m2 = np.mean(np.array(k))
    m3 = np.mean(np.array(c)/np.array(y))
    m4 = np.std(np.array(y))**2
    m5 = np.corrcoef(np.array(c[0:99]),np.array(c[1:100]))[0,1]
    m6 = np.corrcoef(np.array(c[0:100]), np.array(k[0:100]))[0,1]
    
    m=np.array([m1, m2, m3, m4, m5, m6])    
    
    return m    

In [338]:
data_moments(macro)

array([  9.28179049e+06,   6.64398514e+06,   5.84200000e-01,
         2.83778251e+13,   9.40559181e-01,   9.40803054e-01])

In [366]:
def criterion(params, *args):
    alpha, rho, mu, sigma = params
    xvals, unif_vals, W = args
    err = (model_moments(unif_vals,alpha, rho, mu, sigma)-data_moments(xvals))/data_moments(xvals)
    crit_val = np.dot(np.dot(err.T, W), err)
    
    return crit_val

In [367]:
params = np.array([ 0.43,  0.9 ,  9.2 ,  0.02])
W = np.eye(6)
smm_args = (macro, unif_vals, W)
criterion(params,macro, unif_vals, W)

1.8521499313374798

In [371]:
alpha_init = 0.47
rho_init = 0.9
mu_init = 9.2
sigma_init = 0.1

params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
W = np.eye(6)
smm_args = (macro, unif_vals, W)
results = opt.minimize(criterion, params_init, args=(smm_args),
                       method='SLSQP', 
                       bounds=((0.01, 0.99), (-0.99,0.99), (5,14), (0.01, 1.1)))
alpha_SMM, rho_SMM, mu_SMM, sigma_SMM = results.x
print('alpha_SMM=', alpha_SMM, ' rho_SMM=', rho_SMM, ' mu_SMM=', mu_SMM, ' sigma_SMM=', sigma_SMM)

alpha_SMM= 0.421058336986  rho_SMM= 0.924230727468  mu_SMM= 9.93169889444  sigma_SMM= 0.0885010843182


### Estimated coefficints

In [375]:
print('alpha_SMM=', alpha_SMM, ' rho_SMM=', rho_SMM, ' mu_SMM=', mu_SMM, ' sigma_SMM=', sigma_SMM)

alpha_SMM= 0.421058336986  rho_SMM= 0.924230727468  mu_SMM= 9.93169889444  sigma_SMM= 0.0885010843182


### Optimal criterion value

In [374]:
results

     fun: 4.272609070082227e-06
     jac: array([ 0.00773056, -0.00125722,  0.00047585, -0.00250992])
 message: 'Optimization terminated successfully.'
    nfev: 141
     nit: 21
    njev: 21
  status: 0
 success: True
       x: array([ 0.42105834,  0.92423073,  9.93169889,  0.08850108])

### Compare data moments and model moments

In [376]:
print('Data moments=', data_moments(macro))
print('Simulated moments=', model_moments(unif_vals, alpha_SMM, rho_SMM, mu_SMM, sigma_SMM))

Data moments= [  9.28179049e+06   6.64398514e+06   5.84200000e-01   2.83778251e+13
   9.40559181e-01   9.40803054e-01]
Simulated moments= [  9.28972122e+06   6.64000131e+06   5.83186924e-01   2.83761068e+13
   9.40702379e-01   9.40439938e-01]
